In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
# Create our features
feature_df = pd.get_dummies(df, columns=["home_ownership","verification_status","issue_d",
                                "pymnt_plan", "hardship_flag", "debt_settlement_flag", 
                                         "initial_list_status", "next_pymnt_d", 
                                         "application_type"])

X = feature_df.drop(columns="loan_status")

# Create our target
y =  df["loan_status"]

In [7]:
# Check the balance of our target values
y.value_counts()

loan_status
low_risk     68470
high_risk      347
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape
Counter(y_train)

Counter({'low_risk': 51352, 'high_risk': 260})

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [9]:
from sklearn.preprocessing import StandardScaler

#First create standard scalar
scaler = StandardScaler()

#Fit scalar to training data
X_scaler = scaler.fit(X_train)

#Scale data
X_train_scale = X_scaler.transform(X_train)
X_test_scale = X_scaler.transform(X_test)

In [10]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rand_forest_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1) 

#Fit the model
rand_forest_model = rand_forest_model.fit(X_train_scale, y_train)

In [11]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rand_forest_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1) 

#Fit the model
rand_forest_model = rand_forest_model.fit(X_train_scale, y_train)

In [12]:
# Calculated the balanced accuracy score
# Making predictions using the testing data.
from sklearn.metrics import accuracy_score

predictions = rand_forest_model.predict(X_test_scale)

# Calculating the accuracy score.
accurate_score = accuracy_score(y_test, predictions)
accurate_score

0.8877651845393781

In [13]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

con_mat= confusion_matrix(y_test, predictions)

con_mat_df = pd.DataFrame(
    con_mat, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)

con_mat_df


,Predicted High Risk,Predicted Low Risk
Actual High Risk,59,28
Actual Low Risk,1903,15215


In [14]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.68      0.89      0.06      0.78      0.59        87
   low_risk       1.00      0.89      0.68      0.94      0.78      0.62     17118

avg / total       0.99      0.89      0.68      0.94      0.78      0.62     17205



In [15]:
# List the features sorted in descending order by feature importance

# Get feature importances from the model
feature_importances = rand_forest_model.feature_importances_

# Get the feature names from the training data
feature_names = X_train.columns

# Create a list of tuples, where each tuple contains a feature name and its importance
features_importance = sorted(zip(feature_names, feature_importances), key=lambda x: x[1], reverse=True)

# Print the sorted feature importances
for feature, importance in features_importance:
    print(f"{feature}: ({importance})")


total_rec_prncp: (0.06960644414860838)
total_pymnt: (0.06699503615040067)
last_pymnt_amnt: (0.06213990295327954)
total_pymnt_inv: (0.05703694960378033)
total_rec_int: (0.052270035291436354)
int_rate: (0.025355289144477468)
issue_d_Jan-2019: (0.019247877968571785)
max_bal_bc: (0.01817900907731312)
out_prncp_inv: (0.017524409701876367)
installment: (0.016359227494961763)
dti: (0.01632771758647055)
mo_sin_old_il_acct: (0.015972739451199248)
mths_since_recent_inq: (0.015434210329092524)
mths_since_rcnt_il: (0.01538345741149803)
annual_inc: (0.015245724461394786)
il_util: (0.015167606539366274)
out_prncp: (0.014690968526685161)
mo_sin_old_rev_tl_op: (0.01456637299632499)
total_rev_hi_lim: (0.013731232681265318)
bc_util: (0.013650967422793616)
avg_cur_bal: (0.013616891854776984)
tot_cur_bal: (0.013248779331435956)
mths_since_recent_bc: (0.01321976557126906)
bc_open_to_buy: (0.013119073989661252)
total_bc_limit: (0.013099060998813663)
revol_bal: (0.012898690328231996)
loan_amnt: (0.0128677195

### Easy Ensemble AdaBoost Classifier

In [16]:
from sklearn.preprocessing import StandardScaler

#First create standard scalar
scaler = StandardScaler()

#Fit scalar to training data
X_scaler = scaler.fit(X_train)

# Scale the data.
X_train_scale = X_scaler.transform(X_train)
X_test_scale = X_scaler.transform(X_test)

In [17]:
from imblearn.ensemble import EasyEnsembleClassifier

ensemble_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ensemble_model = ensemble_model.fit(X_train_scale, y_train)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score
prediction = ensemble_model.predict(X_test_scale)
accurate_score = accuracy_score(y_test, predictions)
accurate_score

0.8877651845393781

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_predict = ensemble_model.predict(X_test_scale)
con_mat = confusion_matrix(y_test, y_predict)

con_mat_df = pd.DataFrame(
    con_mat, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)

con_mat_df

,Predicted High Risk,Predicted Low Risk
Actual High Risk,76,11
Actual Low Risk,1062,16056


In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_predict)) 


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.87      0.94      0.12      0.91      0.81        87
   low_risk       1.00      0.94      0.87      0.97      0.91      0.82     17118

avg / total       0.99      0.94      0.87      0.96      0.91      0.82     17205

